In [1]:
%reset -f
!pyclean .

Cleaning directory .
Total 2 files, 1 directories removed.


In [2]:
from utils import *
%matplotlib inline

In [3]:
no_bg_meta = pd.read_csv(os.path.join(DATA_DIRECTORY,"no_background","no_background.csv"),index_col="image_id").sort_index()
spacecraft_image = [os.path.join(DATA_DIRECTORY,"no_background", "images", i) + ".png" for i in no_bg_meta.index][:40]
spacecraft_bbox = [[k["xmin"], k["ymin"], k["xmax"], k["ymax"]] for i,k in no_bg_meta.iterrows()][:40]
spacecraft_classification = [k["spacecraft_id"]for i,k in no_bg_meta.iterrows()]
spacecraft_mask = [os.path.join(DATA_DIRECTORY,"no_background", "images", i) + ".png" for i in no_bg_meta.index]

In [4]:
model = create_bbox_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      207392    
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 16)      4624      
                                                                 
 conv2d_2 (Conv2D)           (None, 224, 224, 8)       1160      
                                                                 
 flatten (Flatten)           (None, 401408)            0         
                                                                 
 dense (Dense)               (None, 256)               102760704 
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 4)                 5

In [5]:
dataset = bbox_fpn_datasets(spacecraft_image, spacecraft_bbox, batch_size=2)


In [6]:
image_sample, bbox_sample = next(dataset.__iter__())

In [7]:
image_sample.shape

TensorShape([2, 224, 224, 720])

In [8]:
bbox_sample

<tf.Tensor: shape=(2, 4), dtype=float64, numpy=
array([[141.925  ,  67.8125 , 173.775  ,  94.28125],
       [ 30.8    ,  17.0625 ,  73.15   ,  81.8125 ]])>

### train shape is correct

In [9]:
model.compile(
    optimizer='adam', 
    loss='mean_squared_error',
)

In [11]:
model.fit(dataset, epochs=1)

10/10 [==============================] - 5s 146ms/step - loss: 15427.4248


### I only choose 40 images to check whether it works or not, if my GPU allow my to train many epochs and the full dataset lol. `it works at last`